In [1]:
def fit_column_scaler_min_max(minval, maxval, dataframe):
    
    columns = dataframe.columns
    
    scaler = { column : MinMaxScaler(feature_range=(minval, maxval))
              for column in columns}
    
    scaler_fit = [scaler[column].fit_transform(dataframe[[column]])
                  for column in columns]
    
    for idx, column in enumerate(columns):
        dataframe[column] = scaler_fit[idx]
        
    return dataframe, scaler

### Ideias para dataviz:
- [ ] Olhar vendas/receita de categórias por cidade.
- [ ] Agrupar as lojas por vendas, por receita e por cidade.

Explicação boa: https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues/140579#140579

- https://www.kaggle.com/arthurtok/interactive-intro-to-dimensionality-reduction
- http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html
- http://fabian-kostadinov.github.io/2015/05/31/pca-in-r/

[1](https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/) <- Ideia pra usar Knn

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

path = 'data'

train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz').set_index('ID')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')
geo = pd.read_csv(path + '/geo_shop.csv')

In [6]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = pd.merge(shops, geo, left_on='city', right_on='name').fillna(0)
shops.drop(['name'], axis=1, inplace=True)
items_cat['split'] = items_cat['item_category_name'].str.split('-')
items_cat['type'] = items_cat['split'].map(lambda x: x[0].strip())
items_cat['type_code'] = LabelEncoder().fit_transform(items_cat['type'])
items_cat['subtype'] = items_cat['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
items_cat['subtype_code'] = LabelEncoder().fit_transform(items_cat['subtype'])
train['is_year'] = 0
train['is_year'][(train.date_block_num + 1) % 12 == 0]  = 1
train['is_semester'] = 0
train['is_semester'][(train.date_block_num + 1) % 6 == 0]  = 1
train['revenue'] = train['item_price'] * train['item_cnt_day']
col = ['date_block_num', 'item_price', 'item_cnt_day', 'revenue', 'shop_id']
group_index = ['date_block_num', 'shop_id']
aux = train[col].groupby(group_index).mean().reset_index()
aux.rename(columns={'item_cnt_day':'item_cnt-(mean_shop)',
                    'revenue':'revenue-(mean_shop)', 'item_price':'item_price-(mean_shop)'}, inplace=True)
train = pd.merge(train, aux, on=group_index, how='inner')
col = ['date_block_num', 'item_cnt_day', 'revenue', 'item_id']
group_index = ['date_block_num', 'item_id']
aux = train[col].groupby(group_index).mean()
aux.rename(columns={'item_cnt_day':'item_cnt-(mean_item)',
                    'revenue':'revenue-(mean_item)'}, inplace=True)
train = pd.merge(train, aux, left_on=group_index, right_index=True, how='inner')
del aux
train = train[train.item_price < 100000]
train = train[train.item_cnt_day < 1001]
train.drop(484683, axis=0, inplace=True)
train.drop(['date'], axis=1, inplace=True)

/home/ronaldo/anaconda3/envs/Data_Science/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/ronaldo/anaconda3/envs/Data_Science/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [11]:
df_clean = pd.merge(train, shops, on=['shop_id'], how='left')
df_clean = pd.merge(df_clean, items, on=['item_id'], how='left')
df_clean = pd.merge(df_clean, items_cat, on=['item_category_id'], how='left')

df_clean.drop(['shop_name', 'item_name', 'city',
         'item_category_name', 'split',
         'type', 'subtype'], axis=1, inplace=True)

In [16]:
df_clean.drop(['name_PT-BR'], axis=1, inplace=True)

In [17]:
scaler = StandardScaler().fit(df_clean) # salva a média e std de cada coluna, para poder ser aplicado no set de treino.
X_stand = scaler.transform(df_clean)
print('\nMean:', scaler.mean_, '\n\nStd:', scaler.scale_)

/home/ronaldo/anaconda3/envs/Data_Science/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)



Mean: [1.45699100e+01 3.30017433e+01 1.01972264e+04 8.90749155e+02
 1.24190267e+00 9.33397051e-02 1.74211794e-01 1.15762753e+03
 8.90852783e+02 1.24263983e+00 1.15773021e+03 1.24260108e+00
 1.15762774e+03 1.61706946e+01 5.45799630e+01 4.88840415e+01
 4.00013758e+01 9.87235911e+00 2.09608181e+01] 

Std: [9.42298253e+00 1.62269693e+01 6.32429834e+03 1.72049031e+03
 2.29296282e+00 2.90907897e-01 3.79291504e-01 5.68078395e+03
 2.57560493e+02 2.37435938e-01 5.14678854e+02 1.00977738e+00
 3.51834410e+03 7.55497815e+00 9.03680587e+00 2.68467832e+01
 1.71007397e+01 3.49971607e+00 2.08045302e+01]


/home/ronaldo/anaconda3/envs/Data_Science/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [20]:
pca = PCA(n_components=18)
principalComponents = pca.fit_transform(X_stand)
principalDf = pd.DataFrame(data = principalComponents)

In [21]:
pca.explained_variance_ratio_

array([0.1727678 , 0.13893369, 0.11218975, 0.10402218, 0.08239019,
       0.07083186, 0.05980224, 0.05460108, 0.03961425, 0.03837423,
       0.0323635 , 0.02522253, 0.02015705, 0.01923011, 0.0127111 ,
       0.00958972, 0.00315484, 0.00227037])

In [27]:
np.sum(pca.explained_variance_ratio_[:-3])

0.9832115393882968

In [33]:
principalDf.shape, df_clean.shape

((2935846, 18), (2935846, 19))

In [42]:
principalComponents[:,0]

(2935846,)

In [47]:
for lab, col in zip(df_clean.columns,
                        ('blue', 'red', 'green')):
        plt.scatter(principalDf[y==lab, 0],
                    principalDf[y==lab, 1],
                    label=lab)

NameError: name 'y' is not defined

In [31]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,11))

ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=principalDf[0], ys=principalDf[1], zs=principalDf[2], c=df_clean['city_code'], s=50, cmap='Set1')
ax.view_init(0, 35)
plt.xlim(-1, 10)
plt.ylim(0, 4)

(0, 4)

Error in callback <function flush_figures at 0x7f2b45ff5840> (for post_execute):


KeyboardInterrupt: 

In [28]:
df_clean.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,is_year,is_semester,revenue,item_price-(mean_shop),item_cnt-(mean_shop),revenue-(mean_shop),item_cnt-(mean_item),revenue-(mean_item),city_code,latitude,longitude,item_category_id,type_code,subtype_code
0,0,59,22154,999.0,1.0,0,0,999.0,820.174553,1.092041,884.369789,1.0,999.0,30,57.626076,39.884472,37,11,1
1,0,25,22154,999.0,1.0,0,0,999.0,672.071345,1.178454,771.263539,1.0,999.0,13,55.755871,37.617680,37,11,1
2,0,25,22154,999.0,1.0,0,0,999.0,672.071345,1.178454,771.263539,1.0,999.0,13,55.755871,37.617680,37,11,1
3,0,25,22154,999.0,1.0,0,0,999.0,672.071345,1.178454,771.263539,1.0,999.0,13,55.755871,37.617680,37,11,1
4,0,25,22154,999.0,1.0,0,0,999.0,672.071345,1.178454,771.263539,1.0,999.0,13,55.755871,37.617680,37,11,1
